# Imports

In [1]:
import Pkg; Pkg.add("CUDA")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/mnt/storage/home/araising/.julia/environments/v1.7/Project.toml`
  No Changes to `/mnt/storage/home/araising/.julia/environments/v1.7/Manifest.toml`


In [2]:
using Flux, Statistics
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, logitcrossentropy, throttle, @epochs
using Base.Iterators: repeated
using Parameters: @with_kw
using CUDA
using MLDatasets

# Getting the data

In [3]:
using(MLDatasets)

In [18]:
xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
xtest, ytest = MLDatasets.MNIST.testdata(Float32)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [5]:
size(xtest)

(28, 28, 10000)

In [6]:
size(xtrain)

(28, 28, 60000)

# Preprocessing the Data

In [19]:
# Reshape Data in order to flatten each image into a linear array
xtrain = Flux.flatten(xtrain)
xtest = Flux.flatten(xtest)

784×10000 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [20]:
size(xtrain)

(784, 60000)

In [21]:
# One-hot-encode the labels
ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

(Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [22]:
train_data = DataLoader((xtrain, ytrain), batchsize=128, shuffle=true)

DataLoader{Tuple{Matrix{Float32}, OneHotArrays.OneHotMatrix{UInt32, 10, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0]), 128, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG())

In [23]:
test_data = DataLoader((xtest, ytest), batchsize=128, shuffle=true)

DataLoader{Tuple{Matrix{Float32}, OneHotArrays.OneHotMatrix{UInt32, 10, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]), 128, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG())

# Model

In [24]:
imgsize=(28,28,1)
nclasses=10
model = Chain(Dense(prod(imgsize), 100, relu),
    Dense(100, nclasses))

Chain(
  Dense(784 => 100, relu),              # 78_500 parameters
  Dense(100 => 10),                     # 1_010 parameters
)                   # Total: 4 arrays, 79_510 parameters, 310.836 KiB.

In [25]:
## Default is already (0.9, 0.999)
opt = ADAM(1e-4)

Adam(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [26]:
function accuracy(data_loader, model)
    acc = 0
    for (x,y) in data_loader
        acc += sum(onecold(cpu(model(x))) .== onecold(cpu(y)))*1 / size(x,2)
    end
    acc/length(data_loader)
end

accuracy (generic function with 1 method)

In [28]:
function evalcb()
    @show(loss(xtest, ytest))
    @show(accuracy(train_data, model))
    @show(accuracy(test_data, model))
end
throttled_cb = throttle(evalcb, 1)
loss(x,y) = logitcrossentropy(model(x), y)
params = Flux.params(model)

for epoch in 1:10
    @time begin
        Flux.train!(loss, params, train_data, opt, cb=throttled_cb) 
    end
end

loss(xtest, ytest) = 2.3485696f0
accuracy(train_data, model) = 0.11077980632551528
accuracy(test_data, model) = 0.1116495253164557
loss(xtest, ytest) = 0.5250143f0
accuracy(train_data, model) = 0.8709965796019901
accuracy(test_data, model) = 0.8805379746835443
  1.931195 seconds (418.20 k allocations: 1.199 GiB, 2.02% gc time, 13.77% compilation time)
loss(xtest, ytest) = 0.35263696f0
accuracy(train_data, model) = 0.9029184434968017
accuracy(test_data, model) = 0.9087223101265823
  1.258856 seconds (79.71 k allocations: 927.391 MiB, 1.23% gc time)
  0.958958 seconds (53.62 k allocations: 644.371 MiB, 1.19% gc time)
loss(xtest, ytest) = 0.29911953f0
accuracy(train_data, model) = 0.9152063343994313
accuracy(test_data, model) = 0.9186115506329114
  1.258197 seconds (79.70 k allocations: 927.390 MiB, 1.31% gc time)
loss(xtest, ytest) = 0.2695472f0
accuracy(train_data, model) = 0.9243958777540867
accuracy(test_data, model) = 0.9269185126582279
  1.227941 seconds (79.81 k allocations: 927.39

In [29]:
# Final accuracy:
@show accuracy(train_data, model)
@show accuracy(test_data, model)

accuracy(train_data, model) = 0.9483164534470505
accuracy(test_data, model) = 0.9467958860759493


0.9467958860759493